In [1]:
import chromadb
from chromadb.api.types import Embeddings, Metadata
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from chromadb.config import Settings

In [2]:
api = chromadb.Client(
    Settings(chroma_db_impl="postgres", postgres_username="postgres", 
                                postgres_password="postgres", 
                                postgres_hostname="localhost", 
                                postgres_port="5445", 
                                postgres_databasename="postgres")
    )

In [3]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings:
        # embed the documents somehow
        return [[1, 2, 3, 4, 5] for _ in texts]


In [4]:
collection = api.create_collection("all-my-documents", get_or_create=True)

In [6]:
collection.modify(name="updated-documents-6")

In [5]:
# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=["This is document1", "This is document2"], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on these!
    ids=["doc1", "doc2"], # unique for each doc
)

[('ef72bdc8-be14-449d-a643-0528144a3cbc', 'all-my-documents', None, None)]
[('ef72bdc8-be14-449d-a643-0528144a3cbc', 'all-my-documents', None, None)]
